# Heuristic User Guide

A walkthrough of the heuristic functions I've created using Holiday's stroke error scoring and how to understand them.

The goal is to create a function that returns as high a score as possible for any given gene-archetype pairing without sacrificing speed. Aim to:
- Reduce calls to the Stylus API (meaning generate less stroke orders to score)
- Reduce time complexity (meaning reduce the amount of potential stroke maps to iterate through)

In [1]:
import numpy as np

Get used to working with NumPy arrays instead of standard Python lists. When building heuristics, avoid iteration (especially nested for loops) as that makes time complexity worse. Instead, try to vectorize where you can. NumPy has plenty of functions to work with arrays that are faster than Python's built-in methods.

In [2]:
from score_strokes import strokeErrorMatrix
from xmlparse import loadGeometryBases, loadRef, getXmlScore, minXml

2024-06-24T22:50:03.915338Z [INFO ] Stylus initialized - Stylus 1.5.0 [RELEASE - May 21 2024 14:06:24] (c) 2006-2009 Biologic Institute


These are functions from Holiday's code that my heuristics are built from. They are the foundation for building heuristics that I used. Make sure to read over her documentation on her repo, which is linked in the README of this repo.

When first importing you should see a message that says [INFO ] Stylus initialized - Stylus...
This message just means that Stylus is up and running correctly. Make sure that Stylus is configured correctly or else you will encounter errors.

**strokeErrorMatrix** is used to generate an n*m matrix of stroke errors where n is the number of archetype strokes and m is the number of gene strokes.
- In certain genes, n does not match m despite the gene being a mutation of an archetype with m strokes. These are called marks.
- The smaller the stroke error, the closer the gene stroke is to the archetype stroke.
- The stroke error measurement is not perfect and can be improved, which I heavily recommend future project members to focus on.

**loadGeometryBases** loads the gene data for each file in a specified directory.

**loadRef** loads the archetype data for a specific reference character.

**minXml** generates the XML data necessary for Stylus to score a gene.

**getXmlScore** calls the Stylus API and returns a score.

In [3]:
from pathlib import Path

In [4]:
ref_dir = f'{Path.home()}/Stylus_Scoring_Generalization/Reference' # archetype directory
data_dir = f'{Path.home()}/Stylus_Scoring_Generalization/NewGenes' # gene directory
ref_char = "4EFB"

Your reference directory (where to find the archetype data) and your gene directory (where to find the gene data). You can find sample genes (keep in mind these sample genes all have six strokes) in Holiday's repo at Genes/sixgenes, I changed the directory for my own purposes.

The reference character is the Unicode representation of your archetype that the genes will be scored against (see the Reference folder in Holiday's repo).

In [5]:
stroke_count = 6
stroke_map = np.empty(stroke_count, dtype=int)

Before getting into the actual heuristic algorithms it may be helpful to understand the fundamentals. The goal is to generate a stroke map, but what is a stroke map?

A stroke map is an array matching gene strokes to archetype strokes. Stylus isn't able to determine the best match between each gene stroke and each archetype stroke, so we have to do it ourselves. For example, say I'm attempting to match a six stroke gene to a six stroke archetype. Remember that array indices begin from 0. Let's say gene stroke 0 matches up best with archetype stroke 2. The gene stroke becomes the index and the archetype stroke becomes the value in the stroke map array. In this case, stroke_map[0] = 2.

In [6]:
stroke_map[0] = 2

Continue on and you might end up with something like this:

In [7]:
# pretend our heuristic generates this stroke map...
stroke_map[5] = 5
stroke_map[3] = 0
stroke_map[1] = 1
stroke_map[2] = 4
stroke_map[4] = 3

print(stroke_map)

[2 1 4 0 3 5]


Again, the gene strokes are the indices and the archetype strokes are the values. This can be quite confusing but there's no way around it.

In **strokeErrorMatrix**, the rows are the archetype strokes and the columns are the gene strokes. Each row-column coordinate represents the error between that archetype stroke and that gene stroke. So matrix[2][0] represents the error between archetype stroke 2 and gene stroke 0.

In [8]:
from itertools import permutations

def heuristic_total(strokes, ref, p_strokes, p_ref):
    error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref) # Retrieve error matrix
    least = 10000 # Since we want the smallest possible total error, the variable should be set to a high number
    stroke_map = ()
    for priority in permutations(range(len(ref))): # Iterate over every permutation of stroke order
        s = np.sum(error_maps[np.arange(len(error_maps)), priority]) # Sum every error in this particular stroke order
        if s < least: # Check if the generated sum is smaller than the current sum stored
            least = s
            stroke_map = priority
    return np.argsort(stroke_map) # Swap indices and values

As an example, here's one of my heuristic functions. All of my heuristics are located in benchmark.py (which is in this repo). Specifically, this heuristic takes every possible ordering of errors, calculates the sum of the errors, and returns the ordering with the lowest sum.

In [9]:
from compare_genes import getScores

heuristic_scores, heuristic_alignments, marks = getScores(heuristic_total, ref_char, data_dir)
print("Here's one score", heuristic_scores[0])
print("And here's the stroke map that obtained this score", heuristic_alignments[0])

Here's one score 0.2009721093859917
And here's the stroke map that obtained this score [1 2 6 5 3 4]


Holiday's API makes it very simple to obtain scores given a certain heuristic algorithm. Any function with the correct call signature (which can be found in her documentation under **getScores**) is compatible with **getScores**.

Now you might be wondering, why does the stroke map returned from **getScores** range from 1-6 instead of 0-5? Well, that's simply because Stylus starts with stroke 1 and not stroke 0. **getScores** adds 1 to every value in the stroke map before scoring.

But algorithms compatible with **getScores** must return a single stroke order. What if we wanted to score multiple stroke orders and compare them to find the best one?

The problem with doing this is that it likely increases the algorithm's time complexity, which is not ideal. But doing so may yield more accurate scores. In order to approach the problem in this manner, we need to understand what **getScores** is really doing.

In [10]:
# Copied directly from compare_genes.py

def getScoresExample(algorithm, ref_char, data_dir): # changed name to avoid conflicts
    """
    Calculates scores for a set of gene characters against a given reference with a heuristic algorithm
    Files from the directory specified with gene data are read in a deterministic order (same files = same order)
    Input:
    algorithm: A function with the below signature. Takes stroke geometry and archetype geometry and scores the stroke against the archetype
        Input:
        stroke_geometry: List of the strokes from the gene instance
        reference_geometry: List of the strokes from the archetype
        stroke_fractional_distance: List of fractional distances (a.k.a. progress percentages) for each stroke in the gene instance
        reference_fractional_distance: List of fractional distances (a.k.a. progress percentages) for each stroke in the reference
    ref_char: UTF-8 name of the archetype in question
    data_dir: Directory with the gene files for testing
    Output:
    heuristic_scores: The scores returned from the given algorithm for each of the genes in the directory
    heuristic_alignments: Alignments which the algorithm returned
    marks: Whether or not each respective score has a mark, which is an additional stroke which has no counterpart in the archetype
    """
    heuristic_alignments = []
    heuristic_scores = []
    marks = []
    ref_geometry, ref_progress_percentage, output_size = loadRef(ref_char, "Reference")
    g_data, han_chars, base_data, stroke_sets, stroke_orders, f_names = loadGeometryBases(data_dir, output_size)
    for (geometry_length, han_char, bases, stroke_set, stroke_order, f_name) in zip(g_data, han_chars, base_data, stroke_sets, stroke_orders, f_names):
        geometry, progress_percentage = geometry_length
        heuristic_alignment = np.array(algorithm(geometry, ref_geometry, progress_percentage, ref_progress_percentage))+1
        heuristic_alignments.append(heuristic_alignment)
        heuristic_xml = minXml(ref_char, bases, stroke_set, heuristic_alignment)
        heuristic_score = getXmlScore(heuristic_xml)
        heuristic_scores.append(heuristic_score)
        marks.append(len(geometry)!=len(ref_geometry))
    return heuristic_scores, heuristic_alignments, marks

Understanding **getScores** shouldn't be too difficult of a task. You should be familiar with loadRef, loadGeometryBases, minXml, and getXmlScore from Holiday's documentation (right??). It's just a function that loops over each gene file, calls our heuristic algorithm to generate the stroke map for the gene/archetype pairing, and scores the gene based on our stroke map. We can ignore marks for now.

So if we want to score multiple stroke maps, it's simple. We generate XML through **minXml** and call **getXmlScore** for each stroke map.

In [11]:
ref_data = loadRef(ref_char, ref_dir)
char_data = loadGeometryBases(data_dir, ref_data[2])

def heuristic_small(ref_char, ref_data, char_data):
    ref, p_ref, output_size = ref_data
    g_data, _, base_data, stroke_sets, _, _ = char_data # We don't really need han_chars, stroke_orders, or f_names right now
    heuristic_scores = []
    for (geometry_length, bases, stroke_set) in zip(g_data, base_data, stroke_sets):
        strokes, p_strokes = geometry_length
        error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref)
        compare_scores = []
        stroke_maps = []
        smallerrs = np.min(error_maps, axis=1) # Get the smallest error in every row
        smallerr_count = 0
        for priority in permutations(range(len(ref))): # Iterate over every permutation of stroke order
            c = np.count_nonzero(smallerrs == error_maps[np.arange(len(error_maps)), priority]) # Get the frequency of smallest errors in current stroke map
            if c > smallerr_count:
                smallerr_count = c
                stroke_maps.clear() # Remove the stored stroke maps with the previous frequency
                stroke_maps.append(np.argsort(priority))
            elif c == smallerr_count: # Potentially multiple stroke maps with the same frequency
                stroke_maps.append(np.argsort(priority))
        for m in stroke_maps:
            heuristic_xml = minXml(ref_char, bases, stroke_set, m+1) # Remember to add 1 to your stroke maps
            heuristic_score = getXmlScore(heuristic_xml)
            compare_scores.append(heuristic_score)
        heuristic_scores.append(max(compare_scores))
    return heuristic_scores

print("Here's another score", heuristic_small(ref_char, ref_data, char_data)[0])

Here's another score 0.2009721093859917


This is an example of a heuristic that scores multiple stroke maps and returns the heuristic scores for each gene file without using **getScores**. Similar to **heuristic_total**, the heuristic loops over every possible ordering of errors, but this time, it takes a different approach to locating the optimal stroke map. The current heuristic finds the smallest error in each row of the matrix, then counts the number of times these smallest errors appear in the current stroke map. The stroke maps with the greatest number of smallest errors are scored, and the highest scoring stroke map is selected.

Next, we'll move on to benchmarking our heuristics.

In [12]:
import timeit

total_time = timeit.Timer('getScores(heuristic_total, ref_char, data_dir)', globals=locals()).timeit(number=10)
small_time = timeit.Timer('heuristic_small(ref_char, ref_data, char_data)', globals=locals()).timeit(number=10)
formatter = len(max(["heuristic_total", "heuristic_small"], key=len))
print("{:>{}}:\t{} s".format("heuristic_total", formatter, total_time/10))
print("{:>{}}:\t{} s".format("heuristic_small", formatter, small_time/10))

heuristic_total:	0.5946931977756321 s
heuristic_small:	0.4756078031845391 s


When benchmarking code, using timeit yields more accurate results than simply using time.time() for multiple reasons. You can run the function multiple times by changing the number parameter in the Timer.timeit() method. 

**heuristic_small** looks significantly faster than **heuristic_total**, but to make it more fair, we'll move the calls to **loadRef** and **loadGeometryBases** inside the **heuristic_small** function.

In [13]:
from score_strokes import alignStrokes

def heuristic_small_again(ref_char, ref_dir, data_dir):
    ref, p_ref, output_size = loadRef(ref_char, ref_dir)
    g_data, _, base_data, stroke_sets, _, _ = loadGeometryBases(data_dir, output_size) # We don't really need han_chars, stroke_orders, or f_names right now
    heuristic_scores = []
    for (geometry_length, bases, stroke_set) in zip(g_data, base_data, stroke_sets):
        strokes, p_strokes = geometry_length
        error_maps = strokeErrorMatrix(strokes, ref, p_strokes, p_ref)
        compare_scores = []
        stroke_maps = []
        smallerrs = np.min(error_maps, axis=1) # Get the smallest error in every row
        smallerr_count = 0
        for priority in permutations(range(len(ref))): # Iterate over every permutation of stroke order
            c = np.count_nonzero(smallerrs == error_maps[np.arange(len(error_maps)), priority])
            if c > smallerr_count:
                smallerr_count = c
                stroke_maps.clear()
                stroke_maps.append(np.argsort(priority))
            elif c == smallerr_count:
                stroke_maps.append(np.argsort(priority))
        for m in stroke_maps:
            heuristic_xml = minXml(ref_char, bases, stroke_set, m+1) # Remember to add 1 to your stroke maps
            heuristic_score = getXmlScore(heuristic_xml)
            compare_scores.append(heuristic_score)
        heuristic_scores.append(max(compare_scores))
    return heuristic_scores

total_time = timeit.Timer('getScores(heuristic_total, ref_char, data_dir)', globals=locals()).timeit(number=10)
small_time = timeit.Timer('heuristic_small_again(ref_char, ref_dir, data_dir)', globals=locals()).timeit(number=10)
greedy_time = timeit.Timer('getScores(alignStrokes, ref_char, data_dir)', globals=locals()).timeit(number=10) # added on Holiday's greedy algorithm as well
formatter = len(max(["greedy, heuristic_total", "heuristic_small"], key=len))
print("{:>{}}:\t{} s".format("heuristic_total", formatter, total_time/10))
print("{:>{}}:\t{} s".format("heuristic_small", formatter, small_time/10))
print("{:>{}}:\t{} s".format("greedy", formatter, greedy_time/10))

        heuristic_total:	0.5981374118011444 s
        heuristic_small:	0.5815679595340043 s
                 greedy:	0.3878414945676923 s


And that's probably as close as we can get to accurate. But in my personal experience different reference characters and gene files result in different speeds for different heuristics. In real time, 0.6ish seconds doesn't seem bad at all for 62 genes. It's around 0.2 seconds slower than Holiday's original greedy algorithm but much faster than the exhaustive, so it's all good, right?

It's better to think about it this way: my heuristics take a little under double the amount of time as Holiday's greedy algorithm. As the number of genes to be scored increases, the amount of time taken by my heuristics will grow at a faster rate than Holiday's. Which is why increasing the time complexity to achieve more accurate results isn't exactly the best solution, we want to keep things at O(n) or better if possible. There's always improvement to be made!

I've created a script that benchmarks all my heuristic functions at benchmark.py. Adding your own heuristics to benchmark.py isn't hard, just requires a bit of tweaking. Or if you prefer, you can write your own benchmarking program; mine is pretty messy and could definitely be improved on.

One last thing; checking the accuracy of your heuristic. We want our heuristic to generate a stroke map that results in as high of a score as possible, or as close to the exhaustive score as possible. It is quite difficult to get scores that match the exhaustive scores perfectly without sacrificing speed, so we just want to get as close as we can. Here's a comparison of the two heuristics we've been using and Holiday's original greedy algorithm:

In [14]:
total_scores, _, _ = getScores(heuristic_total, ref_char, data_dir)
small_scores = heuristic_small(ref_char, ref_data, char_data)
greedy_scores, _, _ = getScores(alignStrokes, ref_char, data_dir)

total_wins = 0
small_wins = 0
greedy_wins = 0

for (total_score, small_score, greedy_score) in zip(total_scores, small_scores, greedy_scores):
    best_score = max(total_score, small_score, greedy_score)
    if best_score == total_score:
        total_wins += 1
    if best_score == small_score:
        small_wins += 1
    if best_score == greedy_score:
        greedy_wins += 1

print("heuristic_total achieved the highest score", total_wins, "times.")
print("heuristic_small achieved the highest score", small_wins, "times.")
print("greedy achieved the highest score", greedy_wins, "times.")

heuristic_total achieved the highest score 32 times.
heuristic_small achieved the highest score 56 times.
greedy achieved the highest score 40 times.


Good luck building your heuristics! I hope this guide helped.

*Daniel Rhee*